In [11]:
global_sample = False

learning_rate = 0.00008
decay_rate = 0.9
epoch = 50
loss_lambda = 0.1

#NUS81 NUS1k COCO2017
dataSet = "COCO2017"

deeper_tag_model = True
max_length = 5
decay_model = "STEPLR"
step_size = 10

margin_dis = 0.7
loss_lambda = 0.1
hybrid_loss = False

getTenNetFromFile = False
getDecoderFromFile = False
test = False
evalue = True

num_workers = 12

Feature_Dimensions = 1000
Word_Dimensions = 256


model_name = "_lr_" + str(learning_rate)
model_name = model_name + "_dly_" + str(decay_rate)
if deeper_tag_model:
    model_name = model_name + "_deeper"
if global_sample:
    model_name = model_name + "_gs"
model_name = model_name + "_ml_" + str(max_length)
model_name = model_name + "_md_" + str(margin_dis)

print(model_name)

_lr_8e-05_dly_0.9_deeper_ml_5_md_0.7


In [2]:
import torch
import torch.nn.functional as F
import torchvision.models as models
from jupyterplot import ProgressPlot
from tqdm.notebook import tqdm
import numpy as np
import GPUtil
import cv2
import random
from torch.utils.tensorboard import SummaryWriter
from multiprocessing import cpu_count

import NUS_WIDE as nus
import TenNet as TenNet
import PredictionModel as P
from TripletLossFunc import TripletLossFunc
from Word2Vec import *
import COCO as coco

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

if True:
    if device == torch.device('cuda'):
        Gpus = GPUtil.getGPUs()
        for gpu in Gpus:
            print(gpu.memoryTotal)
            if gpu.memoryTotal > 30000:
                batch_size = 256 + 128
                break
            if gpu.memoryTotal > 15000:
                batch_size = 160
                break
            if gpu.memoryTotal > 12000:
                batch_size = 64
                break
            if gpu.memoryTotal > 11000:
                batch_size = 56
                break
            if gpu.memoryTotal > 7000:
                batch_size = 32
                break
                
print("BATCH_SIZE:", batch_size)

16160.0
BATCH_SIZE: 160


In [4]:
if test or device == torch.device('cpu'):
    print("test!!!")
    Number_Of_Images_Train = 100
    Number_Of_Images_Valid = 100
    N_Epochs = 1
    N_Epochs_Decoder = 1

#NUS_WIDE
if dataSet == "NUS81": 
    train_data = nus.NUS_WIDE_Helper(nus.DataSetType.Train_81, min_tag_num=1)
    valid_data = nus.NUS_WIDE_Helper(nus.DataSetType.Test_81, min_tag_num=1)
elif dataSet == "NUS1k":
    BATCH_SIZE = int(BATCH_SIZE/3)
    train_data = nus.NUS_WIDE_Helper(nus.DataSetType.Train_1k, min_tag_num=1)
    valid_data = nus.NUS_WIDE_Helper(nus.DataSetType.Test_1k, min_tag_num=1)
elif dataSet == "COCO2017":
    train_data = coco.coco_Helper(coco.DataSetType.Train17, min_tag_num=1)
    valid_data = coco.coco_Helper(coco.DataSetType.Valid17, min_tag_num=1)


print("number of workers:", num_workers)

train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, drop_last=True, batch_size=batch_size, 
                                           num_workers=num_workers, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(valid_data, shuffle=True, drop_last=True, batch_size=batch_size, 
                                           num_workers=num_workers, pin_memory=True)

train_loader_E = torch.utils.data.DataLoader(train_data, batch_size=int(batch_size/2), num_workers=num_workers, pin_memory=True)
valid_loader_E = torch.utils.data.DataLoader(valid_data, batch_size=int(batch_size/2), num_workers=num_workers, pin_memory=True)


loading annotations into memory...
Done (t=12.24s)
creating index...
index created!
loading annotations into memory...
Done (t=0.38s)
creating index...
index created!
number of workers: 8


In [5]:
if test or device == torch.device('cpu'):
    image_model = TenNet.VGG16_Normalize().to(device)
    static_word_matrix = [[0] * Word_Dimensions for j in range(len(valid_data.get_tag_list()))]
else:
    image_model = TenNet.VGG16_Normalize().to(device)
    static_word_matrix = compute_word_matrix(train_data.image_tags, Word_Dimensions) 
#print(image_model.VGG16.features)
#for param in image_model.VGG16.features.parameters():
#    param.requires_grad = False

In [6]:
triplet_loss = TripletLossFunc(margin_dis, hybrid=hybrid_loss)
print("Margin_Dis", margin_dis, "Loss lambda", loss_lambda)

from torch.utils.tensorboard import SummaryWriter
log_path = "runs/" + model_name
writer = SummaryWriter(log_dir=log_path)

Margin_Dis 0.7 Loss lambda 0.1


In [7]:
tag_list = valid_data.get_tag_list()
filters=[3, 4, 5]
filter_num=[100, 100, 100]
print("filters", filters, "filter_num", filter_num)

tag_model = TenNet.TenNet_Tag(tag_list=tag_list, feature_dims=Feature_Dimensions, word_dims=Word_Dimensions, dropout_prob=0.5, 
                              filters=filters, filter_num=filter_num, deeper=deeper_tag_model, word_matrix=static_word_matrix, 
                              max_length=8).to(device)


filters [3, 4, 5] filter_num [100, 100, 100]


In [8]:

name = "../SavedModelState/IT_model_" + str(margin_dis) + "_bs_" + str(batch_size) + "_lr_" + str(learning_rate) 
name = name + "_g_" + str(decay_rate) + "_L_" + str(loss_lambda) + ".ckpt"

if getTenNetFromFile:
    TenNet.getTenModel(tag_model, image_model, name=name)

else:
    optimizer = torch.optim.RMSprop([{'params' : image_model.parameters()}, 
                                    {'params' : tag_model.parameters()}], lr=learning_rate)
    if decay_model == "STEPLR": 
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=decay_rate)
# run
    TenNet.run(writer, image_model, tag_model, train_loader, valid_loader, train_loader_E, 
                valid_loader_E, triplet_loss, optimizer, scheduler, 
                margin_dis, global_sample, epoch, loss_lambda, print_log=True)
    

  0%|          | 0/50 [00:00<?, ?it/s]

The number of label occurrences in non-zero loss:
[9080, 1011, 4981, 1081, 2109, 1405, 2160, 2772, 1321, 2078, 1107, 1177, 424, 2004, 2549, 3548, 2681, 1198, 1211, 1423, 1407, 893, 1820, 2171, 1269, 993, 1585, 692, 1103, 595, 440, 235, 694, 423, 321, 365, 467, 569, 441, 4768, 1228, 5251, 2252, 2522, 2259, 4844, 1475, 1202, 1572, 1333, 1721, 1417, 624, 1835, 799, 1594, 5896, 2596, 2886, 2618, 6870, 3029, 3044, 2144, 1571, 1394, 1706, 1614, 1216, 2109, 183, 3925, 1689, 3485, 3075, 2935, 628, 1411, 104, 541]
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon'

The number of label occurrences in non-zero loss:
[19511, 1213, 5558, 1370, 2167, 1551, 2436, 2930, 1578, 2155, 1163, 1239, 463, 2526, 2674, 3739, 3104, 1430, 1301, 1542, 1487, 909, 1840, 2259, 1729, 1487, 2273, 1226, 1378, 907, 526, 334, 1022, 803, 579, 638, 758, 1091, 726, 5604, 1467, 6066, 2406, 2827, 2411, 5294, 1748, 1302, 1753, 1410, 1767, 1467, 806, 2282, 1016, 1862, 7489, 3020, 3221, 3042, 8061, 3207, 3562, 2523, 1620, 1771, 1795, 2254, 1291, 2333, 190, 4215, 1899, 3965, 3440, 3089, 705, 1637, 138, 741]
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', '

The number of label occurrences in non-zero loss:
[15560, 1131, 5827, 1212, 2567, 2154, 3003, 3348, 1800, 2378, 1217, 1310, 482, 2270, 2704, 3730, 2908, 1393, 1317, 1587, 1587, 912, 1801, 2249, 1190, 1272, 1568, 624, 1212, 538, 299, 295, 406, 727, 166, 144, 508, 754, 232, 4520, 1095, 4758, 1881, 2204, 1975, 4453, 1551, 1211, 1453, 1341, 1633, 1291, 686, 1854, 872, 1527, 5976, 2556, 2855, 2819, 6428, 2368, 3135, 2214, 1544, 1310, 1702, 1531, 1148, 2084, 173, 3336, 1668, 3476, 3447, 2916, 637, 1527, 108, 525]
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoo

The number of label occurrences in non-zero loss:
[12516, 1048, 5429, 1170, 2298, 1919, 2902, 3204, 1736, 2247, 1191, 1276, 472, 2142, 2660, 3632, 2830, 1298, 1287, 1558, 1554, 908, 1474, 2056, 1010, 1103, 1299, 451, 1135, 480, 193, 187, 322, 496, 118, 112, 389, 506, 159, 3938, 951, 4129, 1576, 1883, 1695, 3983, 1458, 1161, 1270, 1287, 1567, 1241, 603, 1654, 786, 1379, 5249, 2297, 2653, 2621, 5670, 1687, 2724, 2017, 1405, 1134, 1561, 1328, 1064, 1906, 160, 2757, 1499, 3159, 3313, 2751, 611, 1459, 89, 431]
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon'

...

epoch:31:
 1-train dataset train model
loss: 0.076032,  IT_pos_dis: 0.5716,  IT_neg_dis: 1.3673,  II_pos_dis: 0.8390,  II_neg_dis: 1.4358.
 2-valid dataset evalue model
loss: 0.095607,  IT_pos_dis: 0.5715,  IT_neg_dis: 1.3703,  II_pos_dis: 0.8440,  II_neg_dis: 1.4225.
Evaluate result:
Precision: 0.4220,  Recall: 0.5513,  F1: 0.4405,  Accuracy: 0.9576.
Average number of tags
True positive: 1.2659,  Pos. tags prediction: 3.0000,  Pos. tags ground truth (<=3): 2.9249.
Evaluate result:
Precision: 0.4118,  Recall: 0.5297,  F1: 0.4256,  Accuracy: 0.9564.
Average number of tags
True positive: 1.2353,  Pos. tags prediction: 3.0000,  Pos. tags ground truth (<=3): 2.9546.
The number of label occurrences in non-zero loss:
[4563, 637, 2301, 624, 449, 1004, 561, 1508, 823, 920, 605, 579, 257, 1182, 1119, 1065, 1615, 905, 743, 1065, 355, 393, 191, 227, 488, 607, 602, 189, 670, 216, 60, 51, 148, 208, 44, 43, 105, 202, 44, 2080, 525, 2191, 934, 1093, 1024, 2215, 1002, 712, 934, 796, 646, 765, 477, 656

Evaluate result:
Precision: 0.4243,  Recall: 0.5534,  F1: 0.4425,  Accuracy: 0.9578.
Average number of tags
True positive: 1.2729,  Pos. tags prediction: 3.0000,  Pos. tags ground truth (<=3): 2.9249.
Evaluate result:
Precision: 0.4124,  Recall: 0.5312,  F1: 0.4263,  Accuracy: 0.9565.
Average number of tags
True positive: 1.2371,  Pos. tags prediction: 3.0000,  Pos. tags ground truth (<=3): 2.9546.
The number of label occurrences in non-zero loss:
[4465, 618, 2266, 581, 415, 1014, 551, 1494, 806, 924, 624, 574, 258, 1171, 1099, 1076, 1590, 893, 696, 1002, 330, 396, 182, 214, 488, 575, 574, 191, 679, 206, 67, 61, 142, 216, 46, 43, 104, 192, 50, 2037, 506, 2181, 915, 1089, 966, 2169, 1004, 720, 906, 787, 645, 749, 460, 666, 615, 1054, 2461, 1075, 1300, 1140, 3024, 520, 1215, 946, 648, 584, 729, 712, 555, 951, 79, 1071, 797, 1555, 902, 1312, 476, 772, 45, 238]
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sig

Evaluate result:
Precision: 0.4116,  Recall: 0.5275,  F1: 0.4248,  Accuracy: 0.9564.
Average number of tags
True positive: 1.2349,  Pos. tags prediction: 3.0000,  Pos. tags ground truth (<=3): 2.9546.
The number of label occurrences in non-zero loss:
[4395, 606, 2245, 574, 403, 982, 554, 1462, 812, 918, 600, 557, 251, 1098, 1071, 1019, 1581, 880, 705, 1010, 318, 395, 190, 226, 471, 606, 612, 182, 664, 208, 73, 63, 142, 232, 51, 48, 104, 205, 55, 2043, 508, 2073, 909, 1084, 960, 2153, 996, 698, 901, 790, 624, 756, 453, 609, 624, 1048, 2423, 1062, 1287, 1077, 2938, 524, 1195, 947, 645, 563, 701, 709, 557, 970, 78, 1045, 811, 1474, 893, 1297, 470, 746, 37, 235]
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 

The number of label occurrences in non-zero loss:
[4339, 613, 2206, 583, 400, 964, 543, 1425, 767, 926, 586, 539, 249, 1063, 1090, 993, 1499, 851, 683, 998, 326, 386, 197, 237, 467, 595, 600, 184, 658, 192, 65, 54, 133, 212, 37, 39, 105, 180, 51, 1962, 475, 2109, 920, 1099, 945, 2158, 985, 690, 906, 804, 633, 725, 440, 631, 624, 1039, 2404, 1047, 1258, 1099, 2947, 534, 1142, 878, 615, 549, 685, 686, 527, 914, 70, 1034, 774, 1490, 894, 1273, 467, 753, 42, 249]
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orang

KeyboardInterrupt: 